# 1. Import python packages

In [1]:
import pandas as pd
import numpy as np
import os
file_path=r'D:/project_gov_medical/'

# 2. Create Database

In [2]:
import pyodbc
server = 'localhost\SQLEXPRESS' 
username = 'sa' 
password = 'user1' 
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';UID='+username+';PWD='+ password,autocommit=True)
cursor = cnxn.cursor()

sql1='''
USE master;

ALTER DATABASE projectdatahub SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
DROP DATABASE IF EXISTS projectdatahub
create database projectdatahub


'''


cursor.execute(sql1)
cnxn.commit()

cursor.close()

# 3. Establish connection to database

In [3]:
import pyodbc
import urllib
import sqlalchemy



params_datahub = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER=localhost\SQLEXPRESS;"
                                 "DATABASE=projectdatahub;"
                                 "UID=sa;"
                                 "PWD=user1")

engine_datahub = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params_datahub))

# 4. Date Prepartion and ETL Data to datahub

In [5]:
df = pd.read_csv(file_path + 'YR_Data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98311 entries, 0 to 98310
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Case Count               98230 non-null  float64
 1   Gender                   98230 non-null  object 
 2   Age Decade               98229 non-null  object 
 3   Municipality             98230 non-null  object 
 4   Community                98230 non-null  object 
 5   Date Reported            98230 non-null  object 
 6   Estimated Date of Onset  98230 non-null  object 
 7   Acquisition              98230 non-null  object 
 8   Status                   98230 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


In [6]:
'''Change datatype'''
df['Case Count']=df['Case Count'].fillna(0).astype(np.int64)
df['Date Reported']=pd.to_datetime(df['Date Reported'].str.strip(), format='%m/%d/%Y')
df['Estimated Date of Onset'] = pd.to_datetime(df['Estimated Date of Onset'].str.strip(),format='%m/%d/%Y')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98311 entries, 0 to 98310
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Case Count               98311 non-null  int64         
 1   Gender                   98230 non-null  object        
 2   Age Decade               98229 non-null  object        
 3   Municipality             98230 non-null  object        
 4   Community                98230 non-null  object        
 5   Date Reported            98230 non-null  datetime64[ns]
 6   Estimated Date of Onset  98230 non-null  datetime64[ns]
 7   Acquisition              98230 non-null  object        
 8   Status                   98230 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 6.8+ MB


In [8]:
df.to_sql('gov',engine_datahub,if_exists='replace',index=False)

# 5. Append new data to historical data


In [9]:
'''get new data'''
df_new = pd.read_csv(file_path + 'new_data.csv')
df_new.shape

(81, 9)

In [10]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Case Count               81 non-null     int64 
 1   Gender                   81 non-null     object
 2   Age Decade               81 non-null     object
 3   Municipality             81 non-null     object
 4   Community                81 non-null     object
 5   Date Reported            81 non-null     object
 6   Estimated Date of Onset  81 non-null     object
 7   Acquisition              81 non-null     object
 8   Status                   81 non-null     object
dtypes: int64(1), object(8)
memory usage: 5.8+ KB


In [11]:
'''Change datatype'''
df_new['Date Reported']=pd.to_datetime(df_new['Date Reported'].str.strip(), format='%m/%d/%Y')
df_new['Estimated Date of Onset'] = pd.to_datetime(df_new['Estimated Date of Onset'].str.strip(),format='%m/%d/%Y')

In [12]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Case Count               81 non-null     int64         
 1   Gender                   81 non-null     object        
 2   Age Decade               81 non-null     object        
 3   Municipality             81 non-null     object        
 4   Community                81 non-null     object        
 5   Date Reported            81 non-null     datetime64[ns]
 6   Estimated Date of Onset  81 non-null     datetime64[ns]
 7   Acquisition              81 non-null     object        
 8   Status                   81 non-null     object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 5.8+ KB


In [13]:
'''get historical data from source database'''
df_gov = pd.read_sql('gov',engine_datahub)
df_gov.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98311 entries, 0 to 98310
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Case Count               98311 non-null  int64         
 1   Gender                   98230 non-null  object        
 2   Age Decade               98229 non-null  object        
 3   Municipality             98230 non-null  object        
 4   Community                98230 non-null  object        
 5   Date Reported            98230 non-null  datetime64[ns]
 6   Estimated Date of Onset  98230 non-null  datetime64[ns]
 7   Acquisition              98230 non-null  object        
 8   Status                   98230 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 6.8+ MB


In [14]:
'''append new data to historical data'''
df_all = pd.concat([df_gov,df_new])
df_all.shape

(98392, 9)

In [15]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98392 entries, 0 to 80
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Case Count               98392 non-null  int64         
 1   Gender                   98311 non-null  object        
 2   Age Decade               98310 non-null  object        
 3   Municipality             98311 non-null  object        
 4   Community                98311 non-null  object        
 5   Date Reported            98311 non-null  datetime64[ns]
 6   Estimated Date of Onset  98311 non-null  datetime64[ns]
 7   Acquisition              98311 non-null  object        
 8   Status                   98311 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 7.5+ MB


In [16]:
'''drop duplicate'''
df_all.drop_duplicates(keep='first', inplace=True)
df_all.shape

(98312, 9)

In [17]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98312 entries, 0 to 80
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Case Count               98312 non-null  int64         
 1   Gender                   98311 non-null  object        
 2   Age Decade               98310 non-null  object        
 3   Municipality             98311 non-null  object        
 4   Community                98311 non-null  object        
 5   Date Reported            98311 non-null  datetime64[ns]
 6   Estimated Date of Onset  98311 non-null  datetime64[ns]
 7   Acquisition              98311 non-null  object        
 8   Status                   98311 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(6)
memory usage: 7.5+ MB


# 6. Check Missing Value

In [21]:
df_all[df_all.isnull().any(axis=1)]

,Case Count,Gender,Age Decade,Municipality,Community,Date Reported,Estimated Date of Onset,Acquisition,Status
80138,59198,Male,None,Markham,Pending Neighbourhood,2022-01-07,2022-01-05,Under Investigation,Resolved
98230,0,None,None,None,None,NaT,NaT,None,None


In [ ]:
df_all = df_all.drop(98230)

In [35]:
df_all[df_all.isnull().any(axis=1)]

,Case Count,Gender,Age Decade,Municipality,Community,Date Reported,Estimated Date of Onset,Acquisition,Status
80138,59198,Male,None,Markham,Pending Neighbourhood,2022-01-07,2022-01-05,Under Investigation,Resolved


In [36]:
df_all.shape

(98311, 9)

# 7. Export updated data to database

In [37]:
df_all.to_sql('gov',engine_datahub,if_exists='replace',index=False)

In [38]:
pd.read_sql('gov',engine_datahub)

,Case Count,Gender,Age Decade,Municipality,Community,Date Reported,Estimated Date of Onset,Acquisition,Status
0,75762,Female,30s,Richmond Hill,Pending Neighbourhood,2020-02-29,2020-02-25,Workplace Outbreak,Resolved
1,75768,Female,60s,Richmond Hill,Pending Neighbourhood,2020-03-02,2020-02-23,Workplace Outbreak,Resolved
2,75767,Female,70s,Richmond Hill,Pending Neighbourhood,2020-03-03,2020-02-22,Workplace Outbreak,Resolved
3,75765,Male,50s,Vaughan,Maple,2020-03-04,2020-02-27,Workplace Outbreak,Resolved
4,75766,Male,40s,Richmond Hill,Pending Neighbourhood,2020-03-04,2020-02-28,Workplace Outbreak,Resolved
...,...,...,...,...,...,...,...,...,...
98306,38856,Male,Under 20,King,Pending Neighbourhood,2022-03-03,2022-03-03,Under Investigation,Self-Isolating
98307,38860,Male,60s,Vaughan,Maple,2022-03-03,2022-03-03,Under Investigation,Self-Isolating
98308,38871,Male,20s,Richmond Hill,Pending Neighbourhood,2022-03-03,2022-03-03,Under Investigation,Self-Isolating
98309,38872,Male,80s,Markham,Pending Neighbourhood,2022-03-03,2022-03-03,Under Investigation,Self-Isolating


# 7. Publish data to tableau server

In [ ]:
# %pip install pantab
# %pip install numpy --upgrade
# %pip install tableau_api_lib
# %pip install tableauserverclient

In [39]:
import pandas as pd
import numpy as np
# import hvac
import sqlalchemy
import urllib
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from datetime import datetime

#Used for database
import pyodbc

#Used for publishTableau
import pantab
from tableauhyperapi import HyperProcess, Connection, TableDefinition, SqlType, Telemetry, Inserter, CreateMode, TableName
from tableauhyperapi import escape_string_literal
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils.querying import get_projects_dataframe
import tableauserverclient as TSC
import os
file_path=r'D:/project_gov_medical/'
from pathlib import Path

In [40]:
#Script configuration. Generate a UUID and enter it below.
taskID = '001'
taskName = 'PublishTableau - Test'
logID = '' #Default, use for DEV
#logID = '' #Use this one for PRODUCTION
startTime = datetime.now()

server='https://prod-ca-a.online.tableau.com/'
username='mytoken'
token='4JJ+jISDTVqPbsOOM8ppyw==:AoptK4VG3DUJJs1UIvlNRQN1QHe4Te1F'
siteName='carolsspace'
projectName='default'
scr_table='gov'
dest_table='gov'


#===================================================================
#Standard log writer
#Logging function
def writeLog(taskStatus, taskMessage, endTime = None):
    log_engine = connectSQLServer1()
    if endTime == None:
        endTime = datetime.now()
    logColumns = ['TaskID', 'TaskName','Status','Message','StartTime','EndTime']
    df = pd.DataFrame([[taskID, taskName, taskStatus, taskMessage, startTime, endTime]], columns=logColumns)
    df.to_sql('Python_Logging', log_engine, if_exists='append', index=False)

#===================================================================    
#Connection Functions
#SQL Server Connection

def connectSQLServer1():
    SQLConn = "DRIVER={SQL Server Native Client 11.0};""SERVER=localhost\SQLEXPRESS;""DATABASE=projectdatahub;""UID=sa;""PWD=user1"
    SQLparams = urllib.parse.quote_plus(SQLConn)
    SQLEngine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % SQLparams)
    return SQLEngine


#Tableau Connection - Requires pantab, tableauhyperapi, tableau_api_lib, tableauserverclient
def publishTableau(server,token,siteName, projectName, dest_table, df):
    #Save DataFrame output to .hyper file locally
    fileName = str(Path( "__file__" ).parent.absolute())+ '\\' + dest_table + '.hyper'
    pantab.frame_to_hyper(df, fileName, table = dest_table)
    
    #Connect to our Tableau Instance
    tableau_auth = TSC.PersonalAccessTokenAuth(username, token, siteName)
    server = TSC.Server(server)
    server.add_http_options({'verify': False})
    server.use_server_version()

    with server.auth.sign_in(tableau_auth):
        #Define publish mode - Overwrite, Append, or CreateNew
        publishMode = TSC.Server.PublishMode.Overwrite
        
        #Get project_id from project_name
        all_projects, pagination_item = server.projects.get()
        for project in TSC.Pager(server.projects):
            if project.name == projectName:
                project_id = project.id
    
        #Create the datasource object with the project_id
        datasource = TSC.DatasourceItem(project_id)
        
        #Publish datasource
        print(f'{datetime.now()}: Publishing {dest_table} to {projectName}...')
        datasource = server.datasources.publish(datasource, fileName, publishMode)
        print(f'{datetime.now()}: Datasource published. Datasource ID: {format(datasource.id)}')
    
    if os.path.exists(fileName):
        os.remove(fileName)
        print(f'{datetime.now()}: Removed file {fileName} from local')

#===============================================================
try:
    print(f'{startTime}: Script starting.')

    #Example Teradata Connection
    engine = connectSQLServer1()
    queryText2 =f"""
        SELECT * from {scr_table}   
    """
    outputdata = pd.read_sql(queryText2, engine)
    print(outputdata)

    #Example Publish dataframe to Tableau
    publishTableau(server,token,siteName, projectName, dest_table, outputdata)
    
    #Example Teradata Connection
#     engine = connectSQLServer1()
#     queryText2 =f"""
#         SELECT * from PPC   
#     """
#     outputdata = pd.read_sql(queryText2, engine)
#     print(outputdata)

#     #Example Publish dataframe to Tableau
#     publishTableau(siteName, projectName, tablename, outputdata)

    #Write a "pass" status to the log (uncomment writeLog after GUID is entered):
    taskMessage = 'Logging Template Pass'
    writeLog(True, taskMessage) 
    print(f'{datetime.now()}: Done!')

except Exception as e:
    #Write a "fail" status to the log (uncomment writeLog after GUID is entered):
    taskMessage = str(e)
    print(taskMessage)
    #writeLog(False, taskMessage) 
#===================================================================

2022-12-08 10:19:26.486351: Script starting.
       Case Count  Gender Age Decade   Municipality              Community  \
0           75762  Female        30s  Richmond Hill  Pending Neighbourhood   
1           75768  Female        60s  Richmond Hill  Pending Neighbourhood   
2           75767  Female        70s  Richmond Hill  Pending Neighbourhood   
3           75765    Male        50s        Vaughan                  Maple   
4           75766    Male        40s  Richmond Hill  Pending Neighbourhood   
...           ...     ...        ...            ...                    ...   
98306       38856    Male   Under 20           King  Pending Neighbourhood   
98307       38860    Male        60s        Vaughan                  Maple   
98308       38871    Male        20s  Richmond Hill  Pending Neighbourhood   
98309       38872    Male        80s        Markham  Pending Neighbourhood   
98310       38873  Female        20s        Vaughan             Woodbridge   

      Date Reporte